In [78]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.DataFrame()

In [3]:
df['X1'] = [1,2,3,4,5,6,6,7,9,9]
df['X2'] = [5,3,6,8,1,9,5,8,9,2]
df['label'] = [1,1,0,1,0,1,0,1,0,0]

In [4]:
df

,X1,X2,label
0,1,5,1
1,2,3,1
2,3,6,0
3,4,8,1
4,5,1,0
5,6,9,1
6,6,5,0
7,7,8,1
8,9,9,0
9,9,2,0


In [166]:


class AdaBoost:
    def __init__(self):
        pass
    def row_weights(row,alpha):
        if row['label'] == row['y_pred']:
            return row['weight']*np.exp(-alpha)
        else:
            return row['weight']*np.ex(alpha)
    def fit(self,df):
        decision_stumps = list()
        alphas = list()
        for _ in range(3):
            # intilaising weights
            df.shape[0]
            df['weight'] = 1/df.shape[0]
            # creating decision stump
            dt = DecisionTreeClassifier(max_depth=1)
            dt.fit(df[['X1','X2']].values,df[['label']].values)
            # predict the output by decision stump
            df['y_pred'] = dt.predict(df[['X1','X2']].values)
            # calculating error
            error = (df[df['y_pred']!=df['label']]['weight']).values.sum()
            # caluclating weight of stage
            alpha = 0.5*np.log((1-error)/(error+0.00001))
            # print(alpha)
            alphas.append(alpha)
            decision_stumps.append(dt)
            # updating weights
            df['weight_updated'] = df.apply(lambda row : row['weight']*(np.exp(-alpha) if row['label'] == row['y_pred'] else np.exp(alpha)),axis=1)
            sum_weight = df['weight_updated'].sum()
            df['weight_updated'] /= df['weight_updated'].sum() # normalising weights
            df['upper_lmt'] = np.cumsum(df['weight_updated'])
            df['lower_lmt'] = df['upper_lmt'] - df['weight_updated']
            # print(df)
            indexes = []
            for i in range(df.shape[0]):
                val = np.random.random()
                for idx in range(df.shape[0]):
                    # print(idx)
                    if df.loc[idx,'lower_lmt'] <= val < df.loc[idx,'upper_lmt']:
                        indexes.append(idx)
            df = df.loc[indexes,['X1','X2','label']]
            df.reset_index(inplace=True)
        self.decision_stumps_ = decision_stumps
        self.alphas_ = alphas    
        print(alphas)
    def predict(self,x):
        h_x = np.zeros(x.shape[0])
        # print(self.alphas_)
        for alpha,dt in zip(self.alphas_,self.decision_stumps_):
            h_x += alpha*dt.predict(x)
        print(h_x)
        h_x = np.sign(h_x)
        for i in range(h_x.shape[0]):
            if h_x[i] == -1.0:
                h_x[i] = 0
        print(h_x)

In [91]:
 df['weight'] = 1/df.shape[0]

In [92]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(max_depth=1)
dt.fit(df[['X1','X2']].values,df[['label']].values)
df['y_pred'] = dt.predict(df[['X1','X2']].values)

In [93]:
error = (df[df['y_pred']!=df['label']]['weight']).values.sum()

In [94]:
alpha = 0.5*np.log((1-error)/error)
df

,X1,X2,label,weight,y_pred,weight_updated,upper_lmt,lower_lmt
0,1,5,1,0.1,1,0.071429,0.071429,0.000000
1,2,3,1,0.1,1,0.071429,0.142857,0.071429
2,3,6,0,0.1,0,0.166667,0.309524,0.142857
3,4,8,1,0.1,0,0.071429,0.380952,0.309524
4,5,1,0,0.1,0,0.071429,0.452381,0.380952
5,6,9,1,0.1,0,0.071429,0.523810,0.452381
6,6,5,0,0.1,0,0.166667,0.690476,0.523810
7,7,8,1,0.1,0,0.071429,0.761905,0.690476
8,9,9,0,0.1,0,0.166667,0.928571,0.761905
9,9,2,0,0.1,0,0.071429,1.000000,0.928571


In [36]:
df['weight_updated'] = df.apply(lambda row : row['weight']*(np.exp(-alpha) if row['label'] == row['y_pred'] else np.exp(alpha)),axis=1)

In [39]:
df['weight_updated'] /= df['weight_updated'].sum()

In [42]:
df['upper_lmt'] = np.cumsum(df['weight_updated'])

In [44]:
df['lower_lmt'] = df['upper_lmt'] - df['weight_updated']

In [66]:
indexes = []
for i in range(df.shape[0]):
    val = np.random.random()
    for idx in range(df.shape[0]):
        # print(idx)
        if df.loc[idx,'lower_lmt'] <= val < df.loc[idx,'upper_lmt']:
            indexes.append(idx)
indexes

[6, 2, 1, 8, 4, 4, 2, 5, 6, 7]

In [56]:
alpha

0.4236489301936017

In [168]:
adaboost = AdaBoost()
adaboost.fit(df[['X1','X2','label']])
adaboost.predict(df[['X1','X2']].values)

[0.4236322638047066, 1.0985622911679431, 5.756462732485114]
[0.42363226 0.42363226 0.42363226 1.52219455 0.         1.52219455
 0.42363226 1.52219455 1.52219455 0.        ]
[1. 1. 1. 1. 0. 1. 1. 1. 1. 0.]
